In [192]:
import pandas as pd
from sklearn import linear_model
import numpy as np
import re

# Migration

In [193]:
# migr = immigration
migr_suffix = 'COMPLET,TOTAL,NR,T,'
gdp_suffix = 'CLV10_EUR_HAB,B1GQ,'
unempl_suffix = 'TOTAL,PC_ACT,T,'

dict_countries = {
    'AL': 'Albania',#
    'AT': 'Austria',
    'BA': 'Bosnia and Herzegovina',#
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CH': 'Switzerland',
    'CY': 'Cyprus',
    'CZ': 'Czechia',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EL': 'Greece',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'IE': 'Ireland',
    'IS': 'Iceland',
    'IT': 'Italy',
    'LI': 'Liechtenstein',
    'LT': 'Latvia',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'ME': 'Montenegro',
    'MK': 'North Macedonia',#
    'MT': 'Malta',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'RS': 'Serbia',#
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'TR': 'Turkey',
    'UK': 'United Kingdom'
}

def translate_countries(shortcut):
    shortcut_cut = shortcut.replace(suffix, '')
    return dict_countries[shortcut_cut]

In [194]:
migr = pd.read_csv('immigration.tsv', sep='\t')

In [195]:
migr.head()

,"agedef,age,unit,sex,geo\time",2017,2016,2015,2014,2013,2012,2011,2010,2009,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,"COMPLET,TOTAL,NR,F,AT",52863,59191,69345,52494,47432,42524,38007,33863,33019,...,41346,33793,32392,32883,:,:,:,:,:,:
1,"COMPLET,TOTAL,NR,F,BE",58613,57056,63217,57681,57203,60747,68912 b,64372 b,:,...,41928,38156,29363,30882,31610,32834,30297,31583,31406,29563
2,"COMPLET,TOTAL,NR,F,BG",13126 p,10300,11730,11903 p,8074 p,5921 bp,:,:,:,...,:,:,:,:,:,:,:,:,:,:
3,"COMPLET,TOTAL,NR,F,CH",68334,71356,73714,75173,76841,70586,70755 b,76513,76124,...,53754,47858,46220,48649,58066,64892,71620,75641,77819,72508
4,"COMPLET,TOTAL,NR,F,CY",11316,8829,8688,6448,6675,11681,12707,11494,13026 b,...,:,:,:,:,:,:,:,:,:,:


In [196]:
migr.rename(columns={'agedef,age,unit,sex,geo\\time':'Country'}, inplace=True)

In [197]:
suffix = migr_suffix
migr = migr[migr.Country.str.startswith(suffix)]
migr.head()

,Country,2017,2016,2015,2014,2013,2012,2011,2010,2009,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
64,"COMPLET,TOTAL,NR,T,AT",111801,129509,166323,116262,101866,91557,82230,70978,69295,...,86710,72723,70122,69930,:,:,:,:,:,:
65,"COMPLET,TOTAL,NR,T,BE",126703,123702,146626,123158,120078,129477,147377 b,135281 b,:,...,91624,83812,58849,61522,62950,66147,63749,66763,67460,62662
66,"COMPLET,TOTAL,NR,T,BG",25597 p,21241,25223 p,26615 p,18570 p,14103 bp,:,:,:,...,:,:,:,:,:,:,:,:,:,:
67,"COMPLET,TOTAL,NR,T,CH",143377,149305,153627,156282,160157,149051,148799 b,161778,160623,...,107953,95955,91687,97591,113967,130188,144537,157190,164773,154244
68,"COMPLET,TOTAL,NR,T,CY",21306,17391,15183,9212,13149,17476,23037,20206,22581 b,...,8524,8801,:,:,:,:,:,:,:,:


In [198]:
migr['Country'] = migr['Country'].apply(translate_countries)

In [199]:
migr.sort_values('Country', inplace=True)

In [200]:
cols = list(migr.columns)
cols = [col.replace(' ', '') for col in cols]
migr.columns = cols

In [201]:
def remove_letters_cast_to_int(item):
    return re.sub('\D', '', item)

for col in cols:
    if col != 'Country':
        migr[col] = migr[col].apply(remove_letters_cast_to_int)

In [241]:
migr

,Country,2017,2016,2015,2014,2013,2012,2011,2010,2009,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
64,Austria,111801,129509,166323,116262,101866,91557,82230,70978,69295,...,86710,72723,70122,69930,,,,,,
65,Belgium,126703,123702,146626,123158,120078,129477,147377,135281,,...,91624,83812,58849,61522,62950,66147,63749,66763,67460,62662
66,Bulgaria,25597,21241,25223,26615,18570,14103,,,,...,,,,,,,,,,
77,Croatia,15553,13985,11706,10638,10378,8959,8534,8846,13213,...,32910,51784,52343,44596,42026,33426,57702,48324,10050,
68,Cyprus,21306,17391,15183,9212,13149,17476,23037,20206,22581,...,8524,8801,,,,,,,,
69,Czechia,51847,64083,29602,29897,30124,34337,27114,48317,75620,...,,,12880,10857,10540,,,,,
71,Denmark,68579,74383,78492,68388,60312,54409,52833,52236,51800,...,50236,51372,50105,54445,63187,44961,43400,43377,43567,40715
72,Estonia,17616,14822,15413,3904,4109,2639,3709,2810,3884,...,,,,,,,,,,
75,Finland,31797,34905,28746,31507,31941,31278,29481,25636,26699,...,14744,14192,13564,13294,12222,11611,14795,14554,19001,13558
76,France,369964,378115,364221,340383,338752,327431,319816,307111,296970,...,,,,,,,,,,


# GDP

In [203]:
gdp = pd.read_csv('gdp.tsv', sep='\t')

In [204]:
gdp.head()

,"unit,na_item,geo\time",2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
0,"CLV10_EUR_HAB,B1GQ,AL",:,3670 p,3530,3410,3330,3260,3230,3180,3090,...,:,:,:,:,:,:,:,:,:,:
1,"CLV10_EUR_HAB,B1GQ,AT",37810,37090,36430,36140,36130,36180,36390,36300,35390,...,:,:,:,:,:,:,:,:,:,:
2,"CLV10_EUR_HAB,B1GQ,BE",35670,35330,34810,34470,33960,33590,33610,33590,33330,...,:,:,:,:,:,:,:,:,:,:
3,"CLV10_EUR_HAB,B1GQ,BG",6550 p,6310 p,6050 p,5790 p,5530 p,5400 p,5350 p,5300,5050,...,:,:,:,:,:,:,:,:,:,:
4,"CLV10_EUR_HAB,B1GQ,CH",59870 p,58700 p,58200,57840,57730,57040,56660,56690,56150,...,:,:,:,:,:,:,:,:,:,:


In [205]:
gdp.rename(columns={'unit,na_item,geo\\time':'Country'}, inplace=True)
gdp.head()

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
0,"CLV10_EUR_HAB,B1GQ,AL",:,3670 p,3530,3410,3330,3260,3230,3180,3090,...,:,:,:,:,:,:,:,:,:,:
1,"CLV10_EUR_HAB,B1GQ,AT",37810,37090,36430,36140,36130,36180,36390,36300,35390,...,:,:,:,:,:,:,:,:,:,:
2,"CLV10_EUR_HAB,B1GQ,BE",35670,35330,34810,34470,33960,33590,33610,33590,33330,...,:,:,:,:,:,:,:,:,:,:
3,"CLV10_EUR_HAB,B1GQ,BG",6550 p,6310 p,6050 p,5790 p,5530 p,5400 p,5350 p,5300,5050,...,:,:,:,:,:,:,:,:,:,:
4,"CLV10_EUR_HAB,B1GQ,CH",59870 p,58700 p,58200,57840,57730,57040,56660,56690,56150,...,:,:,:,:,:,:,:,:,:,:


In [206]:
suffix = gdp_suffix
gdp = gdp[gdp.Country.str.startswith(suffix)]
gdp = gdp[~gdp['Country'].str.endswith('EA')]
gdp = gdp[~gdp['Country'].str.endswith('EA12')]
gdp = gdp[~gdp['Country'].str.endswith('EA19')]
gdp = gdp[~gdp['Country'].str.endswith('EU15')]
gdp = gdp[~gdp['Country'].str.endswith('2019')]
gdp = gdp[~gdp['Country'].str.endswith('EU28')]
gdp

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
0,"CLV10_EUR_HAB,B1GQ,AL",:,3670 p,3530,3410,3330,3260,3230,3180,3090,...,:,:,:,:,:,:,:,:,:,:
1,"CLV10_EUR_HAB,B1GQ,AT",37810,37090,36430,36140,36130,36180,36390,36300,35390,...,:,:,:,:,:,:,:,:,:,:
2,"CLV10_EUR_HAB,B1GQ,BE",35670,35330,34810,34470,33960,33590,33610,33590,33330,...,:,:,:,:,:,:,:,:,:,:
3,"CLV10_EUR_HAB,B1GQ,BG",6550 p,6310 p,6050 p,5790 p,5530 p,5400 p,5350 p,5300,5050,...,:,:,:,:,:,:,:,:,:,:
4,"CLV10_EUR_HAB,B1GQ,CH",59870 p,58700 p,58200,57840,57730,57040,56660,56690,56150,...,:,:,:,:,:,:,:,:,:,:
5,"CLV10_EUR_HAB,B1GQ,CY",23770 p,23120,22360,21040,20240,20400,21780,22900,23400,...,:,:,:,:,:,:,:,:,:,:
6,"CLV10_EUR_HAB,B1GQ,CZ",17640,17200,16520,16160,15370,14980,15060,15200,14900,...,:,:,:,:,:,:,:,:,:,:
7,"CLV10_EUR_HAB,B1GQ,DE",35860,35420,34700,34220,33930,33330,33280,33200,31940,...,:,:,:,:,:,:,:,:,:,:
8,"CLV10_EUR_HAB,B1GQ,DK",48260,47360,46720,45630,44890,44410,44170,44240,43840,...,30310,29080,28320,27290,27470,27630,26670,26170,25770,24390
12,"CLV10_EUR_HAB,B1GQ,EE",15090,14440,13650,13330,13060,12640,12430,12010,11150,...,:,:,:,:,:,:,:,:,:,:


In [207]:
gdp['Country'] = gdp['Country'].apply(translate_countries)
gdp.head()

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
0,Albania,:,3670 p,3530,3410,3330,3260,3230,3180,3090,...,:,:,:,:,:,:,:,:,:,:
1,Austria,37810,37090,36430,36140,36130,36180,36390,36300,35390,...,:,:,:,:,:,:,:,:,:,:
2,Belgium,35670,35330,34810,34470,33960,33590,33610,33590,33330,...,:,:,:,:,:,:,:,:,:,:
3,Bulgaria,6550 p,6310 p,6050 p,5790 p,5530 p,5400 p,5350 p,5300,5050,...,:,:,:,:,:,:,:,:,:,:
4,Switzerland,59870 p,58700 p,58200,57840,57730,57040,56660,56690,56150,...,:,:,:,:,:,:,:,:,:,:


In [208]:
gdp.sort_values('Country', inplace=True)

In [209]:
cols = list(gdp.columns)
cols = [col.replace(' ', '') for col in cols]
gdp.columns = cols

In [210]:
def remove_letters_cast_to_int(item):
    return re.sub('\D', '', item)

for col in cols:
    if col != 'Country':
        gdp[col] = gdp[col].apply(remove_letters_cast_to_int)

In [211]:
gdp

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
0,Albania,,3670,3530,3410,3330,3260,3230,3180,3090,...,,,,,,,,,,
1,Austria,37810,37090,36430,36140,36130,36180,36390,36300,35390,...,,,,,,,,,,
2,Belgium,35670,35330,34810,34470,33960,33590,33610,33590,33330,...,,,,,,,,,,
3,Bulgaria,6550,6310,6050,5790,5530,5400,5350,5300,5050,...,,,,,,,,,,
20,Croatia,11940,11500,11050,10590,10290,10260,10280,10480,10480,...,,,,,,,,,,
5,Cyprus,23770,23120,22360,21040,20240,20400,21780,22900,23400,...,,,,,,,,,,
6,Czechia,17640,17200,16520,16160,15370,14980,15060,15200,14900,...,,,,,,,,,,
8,Denmark,48260,47360,46720,45630,44890,44410,44170,44240,43840,...,30310,29080,28320,27290,27470,27630,26670,26170,25770,24390
12,Estonia,15090,14440,13650,13330,13060,12640,12430,12010,11150,...,,,,,,,,,,
18,Finland,36820,36270,35280,34470,34390,34660,35140,35810,35080,...,21040,20490,20000,19510,19350,,,,,


In [234]:
gdp.loc[:, '2018':'2000']

,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,,3670,3530,3410,3330,3260,3230,3180,3090,2960,2850,2630,2460,2310,2180,2060,1940,1850,1700
1,37810,37090,36430,36140,36130,36180,36390,36300,35390,34830,36280,35870,34700,33710,33200,32520,32360,31990,31710
2,35670,35330,34810,34470,33960,33590,33610,33590,33330,32700,33640,33760,32800,32200,31640,30680,30490,30110,29890
3,6550,6310,6050,5790,5530,5400,5350,5300,5050,4990,5140,4820,4500,4190,3890,3640,3440,3230,3010
20,11940,11500,11050,10590,10290,10260,10280,10480,10480,10630,11460,11260,10700,10200,9790,9400,8890,8480,7840
5,23770,23120,22360,21040,20240,20400,21780,22900,23400,23550,24680,24420,23740,23050,22300,21520,21230,20700,20130
6,17640,17200,16520,16160,15370,14980,15060,15200,14900,14610,15430,15190,14460,13570,12780,12180,11760,11540,11170
8,48260,47360,46720,45630,44890,44410,44170,44240,43840,43220,45700,46210,45990,44400,43520,42490,42430,42390,42190
12,15090,14440,13650,13330,13060,12640,12430,12010,11150,10830,12640,13270,12260,11110,10090,9390,8680,8070,7570
18,36820,36270,35280,34470,34390,34660,35140,35810,35080,34150,37330,37210,35490,34250,33440,32250,31690,31230,30510


# Unemployment

In [212]:
unempl = pd.read_csv('unemployment.tsv', sep='\t')
unempl.head()

,"age,unit,sex,geo\time",2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1992,1991,1990,1989,1988,1987,1986,1985,1984,1983
0,"TOTAL,PC_ACT,F,AT",4.7,5.0,5.6,5.3,5.4,5.3,4.8,4.6,4.6,...,:,:,:,:,:,:,:,:,:,:
1,"TOTAL,PC_ACT,F,BE",5.6,7.1 b,7.6,7.8,7.9,8.2,7.4,7.2,8.5,...,10.0,9.8,10.4,11.7,13.7,15.6,:,:,:,:
2,"TOTAL,PC_ACT,F,BG",4.7,6.0,7.0,8.4,10.4,11.8,10.8,10.1,9.6 d,...,:,:,:,:,:,:,:,:,:,:
3,"TOTAL,PC_ACT,F,CY",8.8,11.3,13.4,14.8,15.1,15.2,11.1,7.7,6.4,...,:,:,:,:,:,:,:,:,:,:
4,"TOTAL,PC_ACT,F,CZ",2.8,3.6,4.7,6.1,7.4,8.3,8.2,7.9,8.5,...,:,:,:,:,:,:,:,:,:,:


In [213]:
unempl.rename(columns={'age,unit,sex,geo\\time':'Country'}, inplace=True)
unempl.head()

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1992,1991,1990,1989,1988,1987,1986,1985,1984,1983
0,"TOTAL,PC_ACT,F,AT",4.7,5.0,5.6,5.3,5.4,5.3,4.8,4.6,4.6,...,:,:,:,:,:,:,:,:,:,:
1,"TOTAL,PC_ACT,F,BE",5.6,7.1 b,7.6,7.8,7.9,8.2,7.4,7.2,8.5,...,10.0,9.8,10.4,11.7,13.7,15.6,:,:,:,:
2,"TOTAL,PC_ACT,F,BG",4.7,6.0,7.0,8.4,10.4,11.8,10.8,10.1,9.6 d,...,:,:,:,:,:,:,:,:,:,:
3,"TOTAL,PC_ACT,F,CY",8.8,11.3,13.4,14.8,15.1,15.2,11.1,7.7,6.4,...,:,:,:,:,:,:,:,:,:,:
4,"TOTAL,PC_ACT,F,CZ",2.8,3.6,4.7,6.1,7.4,8.3,8.2,7.9,8.5,...,:,:,:,:,:,:,:,:,:,:


In [214]:
suffix = unempl_suffix
unempl = unempl[unempl.Country.str.startswith(suffix)]
unempl = unempl[~unempl['Country'].str.endswith('EA')]
unempl = unempl[~unempl['Country'].str.endswith('EA18')]
unempl = unempl[~unempl['Country'].str.endswith('EA19')]
unempl = unempl[~unempl['Country'].str.endswith('EU25')]
unempl = unempl[~unempl['Country'].str.endswith('EU27')]
unempl = unempl[~unempl['Country'].str.endswith('2019')]
unempl = unempl[~unempl['Country'].str.endswith('EU28')]
unempl = unempl[~unempl['Country'].str.endswith('JP')]
unempl = unempl[~unempl['Country'].str.endswith('US')]
unempl

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1992,1991,1990,1989,1988,1987,1986,1985,1984,1983
78,"TOTAL,PC_ACT,T,AT",4.9,5.5,6.0,5.7,5.6,5.4,4.9,4.6,4.8,...,:,:,:,:,:,:,:,:,:,:
79,"TOTAL,PC_ACT,T,BE",6.0,7.1 b,7.8,8.5,8.5,8.4,7.6,7.2,8.3,...,7.1,6.4,6.6,7.4,8.8,9.8,:,:,:,:
80,"TOTAL,PC_ACT,T,BG",5.2,6.2,7.6,9.2,11.4,13.0,12.3,11.3,10.3 d,...,:,:,:,:,:,:,:,:,:,:
81,"TOTAL,PC_ACT,T,CY",8.4,11.1,13.0,15.0,16.1,15.9,11.9,7.9,6.3,...,:,:,:,:,:,:,:,:,:,:
82,"TOTAL,PC_ACT,T,CZ",2.2,2.9,4.0,5.1,6.1,7.0,7.0,6.7,7.3,...,:,:,:,:,:,:,:,:,:,:
83,"TOTAL,PC_ACT,T,DE",3.4,3.8,4.1,4.6,5.0,5.2,5.4,5.8,7.0,...,6.6,5.5,:,:,:,:,:,:,:,:
84,"TOTAL,PC_ACT,T,DK",5.1,5.8,6.0,6.3,6.9,7.4,7.8,7.8,7.7,...,8.6,7.9,7.2,6.8,5.7,5.0,5.0,6.7,7.9,8.4
88,"TOTAL,PC_ACT,T,EE",5.4,5.8,6.8,6.2,7.4,8.6,10.0,12.3,16.7,...,:,:,:,:,:,:,:,:,:,:
89,"TOTAL,PC_ACT,T,EL",19.3,21.5,23.6,24.9,26.5,27.5,24.5,17.9,12.7,...,:,:,:,:,:,:,:,:,:,:
90,"TOTAL,PC_ACT,T,ES",15.3,17.2,19.6,22.1,24.5,26.1,24.8,21.4,19.9,...,17.0,15.5,15.5,16.5,18.7,19.7,:,:,:,:


In [215]:
unempl['Country'] = unempl['Country'].apply(translate_countries)

In [216]:
unempl.sort_values('Country', inplace=True)
unempl.head()

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1992,1991,1990,1989,1988,1987,1986,1985,1984,1983
78,Austria,4.9,5.5,6.0,5.7,5.6,5.4,4.9,4.6,4.8,...,:,:,:,:,:,:,:,:,:,:
79,Belgium,6.0,7.1 b,7.8,8.5,8.5,8.4,7.6,7.2,8.3,...,7.1,6.4,6.6,7.4,8.8,9.8,:,:,:,:
80,Bulgaria,5.2,6.2,7.6,9.2,11.4,13.0,12.3,11.3,10.3 d,...,:,:,:,:,:,:,:,:,:,:
97,Croatia,8.4,11.0,13.4,16.1,17.2,17.4,15.8,13.7,11.8,...,:,:,:,:,:,:,:,:,:,:
81,Cyprus,8.4,11.1,13.0,15.0,16.1,15.9,11.9,7.9,6.3,...,:,:,:,:,:,:,:,:,:,:


In [217]:
cols = list(unempl.columns)
cols = [col.replace(' ', '') for col in cols]
unempl.columns = cols

In [218]:
def remove_letters_cast_to_int(item):
#     print(item, type(item))
    if isinstance(item, float):
        return item
    return re.sub('[^\d\.]', '', item)

for col in cols:
    if col != 'Country':
        unempl[col] = unempl[col].apply(remove_letters_cast_to_int)

In [219]:
unempl

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1992,1991,1990,1989,1988,1987,1986,1985,1984,1983
78,Austria,4.9,5.5,6.0,5.7,5.6,5.4,4.9,4.6,4.8,...,,,,,,,,,,
79,Belgium,6.0,7.1,7.8,8.5,8.5,8.4,7.6,7.2,8.3,...,7.1,6.4,6.6,7.4,8.8,9.8,,,,
80,Bulgaria,5.2,6.2,7.6,9.2,11.4,13.0,12.3,11.3,10.3,...,,,,,,,,,,
97,Croatia,8.4,11.0,13.4,16.1,17.2,17.4,15.8,13.7,11.8,...,,,,,,,,,,
81,Cyprus,8.4,11.1,13.0,15.0,16.1,15.9,11.9,7.9,6.3,...,,,,,,,,,,
82,Czechia,2.2,2.9,4.0,5.1,6.1,7.0,7.0,6.7,7.3,...,,,,,,,,,,
84,Denmark,5.1,5.8,6.0,6.3,6.9,7.4,7.8,7.8,7.7,...,8.6,7.9,7.2,6.8,5.7,5.0,5.0,6.7,7.9,8.4
88,Estonia,5.4,5.8,6.8,6.2,7.4,8.6,10.0,12.3,16.7,...,,,,,,,,,,
95,Finland,7.4,8.6,8.8,9.4,8.7,8.2,7.7,7.8,8.4,...,11.7,6.6,3.2,3.1,4.2,,,,,
96,France,9.1,9.4,10.1,10.4,10.3,10.3,9.8,9.2,9.3,...,9.0,8.1,7.9,8.1,8.5,8.9,8.9,8.7,8.4,7.3


In [235]:
unempl.loc[:, '2018':'2000']

,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
78,4.9,5.5,6.0,5.7,5.6,5.4,4.9,4.6,4.8,5.3,4.1,4.9,5.3,5.6,5.5,4.8,4.4,4.0,3.9
79,6.0,7.1,7.8,8.5,8.5,8.4,7.6,7.2,8.3,7.9,7.0,7.5,8.3,8.5,8.4,8.2,7.5,6.6,6.9
80,5.2,6.2,7.6,9.2,11.4,13.0,12.3,11.3,10.3,6.8,5.6,6.9,9.0,10.1,12.1,13.7,18.2,19.6,16.4
97,8.4,11.0,13.4,16.1,17.2,17.4,15.8,13.7,11.8,9.3,8.6,9.9,11.6,13.0,13.8,14.1,15.3,16.0,15.6
81,8.4,11.1,13.0,15.0,16.1,15.9,11.9,7.9,6.3,5.4,3.7,3.9,4.6,5.3,4.6,4.1,3.5,3.9,4.8
82,2.2,2.9,4.0,5.1,6.1,7.0,7.0,6.7,7.3,6.7,4.4,5.3,7.1,7.9,8.3,7.8,7.3,8.1,8.8
84,5.1,5.8,6.0,6.3,6.9,7.4,7.8,7.8,7.7,6.4,3.7,3.8,3.9,4.8,5.5,5.4,4.6,4.5,4.3
88,5.4,5.8,6.8,6.2,7.4,8.6,10.0,12.3,16.7,13.5,5.5,4.6,5.9,8.0,10.1,10.3,11.2,13.0,14.6
95,7.4,8.6,8.8,9.4,8.7,8.2,7.7,7.8,8.4,8.2,6.4,6.9,7.7,8.4,8.8,9.0,9.1,9.1,9.8
96,9.1,9.4,10.1,10.4,10.3,10.3,9.8,9.2,9.3,9.1,7.4,8.0,8.8,8.9,8.9,8.5,7.9,7.8,8.6


# coefficients

In [236]:
countries = list(dict_countries.values())
countries.sort()
countries

['Albania',
 'Austria',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'Ireland',
 'Italy',
 'Latvia',
 'Latvia',
 'Liechtenstein',
 'Luxembourg',
 'Malta',
 'Montenegro',
 'Netherlands',
 'North Macedonia',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'United Kingdom']

In [240]:
print('GDP')
for c in countries:
    print(c)
    cGdp = gdp.loc[gdp['Country'] == c]
    cMig = migr.loc[migr['Country'] == c]
    
    xtrain = []
    ytrain = []
    
    for i in range(2000, 2018):
        print(cGdp.iloc[0][str(i)])
#         xtrain.append(cGdp.iloc[0][str(i)])
#         ytrain.append(cMig.iloc[0][str(i)])
    
#     gdp2007 = cGdp.iloc[0]['2007']
#     gdp2011 = cGdp.iloc[0]['2011']
#     gdp2016 = cGdp.iloc[0]['2016']
#     migr2005 = cMig.iloc[0]['2005']
#     migr2010 = cMig.iloc[0]['2010']
#     migr2015 = cMig.iloc[0]['2015']
    
#     xtrain = np.array([migr2005, migr2010, migr2015]).reshape(-1, 1)
#     ytrain = np.array([gdp2007, gdp2011, gdp2016]).reshape(-1, 1)
    
#     regr = linear_model.LinearRegression()
#     regr.fit(xtrain, ytrain)
#     coef = regr.coef_
#     print(c, ':', coef)

GDP
Albania
1700
1850
1940
2060
2180
2310
2460
2630
2850
2960
3090
3180
3230
3260
3330
3410
3530
3670
Austria
31710
31990
32360
32520
33200
33710
34700
35870
36280
34830
35390
36300
36390
36180
36130
36140
36430
37090
Belgium
29890
30110
30490
30680
31640
32200
32800
33760
33640
32700
33330
33590
33610
33590
33960
34470
34810
35330
Bosnia and Herzegovina


IndexError: single positional indexer is out-of-bounds